In [ ]:
metadata = [
    {
        "user": "john",
        "age": 18,
        "job": "engineer",
        "credit_score": "high",
    },
    {
        "user": "derrick",
        "age": 45,
        "job": "doctor",
        "credit_score": "low",
    },
    {
        "user": "nancy",
        "age": 94,
        "job": "doctor",
        "credit_score": "high",
    },
    {
        "user": "tyler",
        "age": 100,
        "job": "engineer",
        "credit_score": "high",
    },
    {
        "user": "joe",
        "age": 35,
        "job": "dentist",
        "credit_score": "medium",
    },
]
texts = ["foo", "foo", "foo", "bar", "bar"]

In [ ]:
import os
from langchain_aws.embeddings import BedrockEmbeddings

In [ ]:
# create a Titan Embeddings client
embeddings = BedrockEmbeddings()

In [ ]:
from langchain_aws.vectorstores.inmemorydb import InMemoryVectorStore

In [ ]:
endpoint = "<your-cluster-endpoint>"

vds = InMemoryVectorStore.from_texts(
    texts,
    embeddings,
    metadatas=metadata,
    redis_url=f"rediss://{endpoint}:6379/ssl=True&ssl_cert_reqs=none",
    index_name="users",
)

In [ ]:
results = vds.similarity_search("foo")
print(results[0].page_content)

In [ ]:
# return metadata
results = vds.similarity_search("foo", k=3)
meta = results[1].metadata
print("Key of the document in Redis: ", meta.pop("id"))
print("Metadata of the document: ", meta)

In [ ]:
# with scores (distances)
results = vds.similarity_search_with_score("foo", k=5)
for result in results:
    print(f"Content: {result[0].page_content} --- Score: {result[1]}")

In [ ]:
# limit the vector distance that can be returned
results = vds.similarity_search_with_score("foo", k=5, distance_threshold=0.1)
for result in results:
    print(f"Content: {result[0].page_content} --- Score: {result[1]}")

In [ ]:
# limit scores (similarities have to be over .9)
results = vds.similarity_search_with_relevance_scores("foo", k=5, score_threshold=0.9)
for result in results:
    print(f"Content: {result[0].page_content} --- Similarity: {result[1]}")

In [ ]:
# you can also add new documents as follows
new_document = ["baz"]
new_metadata = [{"user": "sam", "age": 50, "job": "janitor", "credit_score": "high"}]
# both the document and metadata must be lists
vds.add_texts(new_document, new_metadata)

In [ ]:
# now query the new document
results = vds.similarity_search("baz", k=3)
print(results[0].metadata)

In [ ]:
# write the schema to a yaml file
vds.write_schema("redis_schema.yaml")

In [ ]:
# now we can connect to our existing index as follows

new_vds = InMemoryVectorStore.from_existing_index(
    embeddings,
    index_name="users",
    redis_url=f"rediss://{endpoint}:6379/ssl=True&ssl_cert_reqs=none",
    schema="redis_schema.yaml",
)
results = new_vds.similarity_search("foo", k=3)
print(results[0].metadata)

In [ ]:
# see the schemas are the same
new_vds.schema == vds.schema

In [ ]:
# create a new index with the new schema
index_schema = {
    "tag": [{"name": "credit_score"}],
    "text": [{"name": "user"}, {"name": "job"}],
    "numeric": [{"name": "age"}],
}

vds, keys = InMemoryVectorStore.from_texts_return_keys(
    texts,
    embeddings,
    metadatas=metadata,
    redis_url=f"rediss://{endpoint}:6379/ssl=True&ssl_cert_reqs=none",
    index_name="users_modified",
    index_schema=index_schema,  # pass in the new index schema
)

In [ ]:

from langchain_aws.vectorstores.inmemorydb import InMemoryDBNum, InMemoryDBTag

# exact matching
has_high_credit = InMemoryDBTag("credit_score") == "high"
does_not_have_high_credit = InMemoryDBTag("credit_score") != "low"


In [ ]:

# numeric filtering
age_is_18 = InMemoryDBNum("age") == 18
age_is_not_18 = InMemoryDBNum("age") != 18
age_is_greater_than_18 = InMemoryDBNum("age") > 18
age_is_less_than_18 = InMemoryDBNum("age") < 18
age_is_greater_than_or_equal_to_18 = InMemoryDBNum("age") >= 18
age_is_less_than_or_equal_to_18 = InMemoryDBNum("age") <= 18


In [ ]:

from langchain_aws.vectorstores.inmemorydb import InMemoryDBFilter

# same examples as above
has_high_credit = InMemoryDBFilter.tag("credit_score") == "high"
does_not_have_high_credit = InMemoryDBFilter.num("age") > 8
job_starts_with_eng = InMemoryDBFilter.text("job") % "eng*"

In [ ]:
from langchain_aws.vectorstores.inmemorydb import InMemoryDBNum

is_over_18 = InMemoryDBNum("age") > 18
is_under_99 = InMemoryDBNum("age") < 99
age_range = is_over_18 & is_under_99
results = vds.similarity_search("foo", filter=age_range)

for result in results:
    print("User:", result.metadata["user"], "is", result.metadata["age"])